## Build ScannerAgent and Messennger Agent

In [1]:
import os 
from dotenv import load_dotenv
import logging 
import requests
from openai import OpenAI

### Internal Classes 
from agents.deals import ScrapedDeal, DealSelection

In [7]:
load_dotenv()
MODEL = "gpt-5-mini"
openai = OpenAI()

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:29<00:00, 29.95s/it]


In [5]:
len(deals)

30

In [6]:
print(deals[10].describe())

Title: Open-Box Samsung Odyssey G5 27" G55C QHD 165Hz Curved Gaming Monitor for $117 + free shipping

Details: Use promo code "VIPOUTLET26NEWY" to drop the price. It's around $180 elsewhere. Buy Now at eBay

Features: 

URL: https://www.dealnews.com/products/Samsung/Samsung-Odyssey-G5-27-G55-C-QHD-165-Hz-Curved-Gaming-Monitor/497271.html?iref=rss-c39


## GPT-5-Mini Parsing -- to summarize deals and identify their price into structured outputs

In [7]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [8]:
def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += "\n\n".join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX

    return user_prompt

In [9]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": make_user_prompt(deals)}
]

In [10]:
response = openai.chat.completions.parse(
    model=MODEL, 
    messages=messages, 
    response_format=DealSelection,
    reasoning_effort="minimal"
)

In [11]:
results = response.choices[0].message.parsed

print(results.deals)

[Deal(product_description="Refurbished Apple iPhone 15 Pro Max (256GB) — a flagship-grade smartphone featuring Apple's latest design and high-end hardware in a refurbished condition. The 256GB capacity provides ample storage for photos, apps, and media, and the Pro Max model includes advanced camera systems, performance-optimized chipset, and a large high-resolution display. This unit is offered as a refurbished device, which typically includes inspection and restoration to working condition and may come with a limited warranty.", price=575.99, url='https://www.dealnews.com/Refurb-Apple-iPhones-at-eBay-Up-to-80-off-free-shipping/21802631.html?iref=rss-c142'), Deal(product_description='EcoFlow Delta 2 Portable Power Station (1,024Wh) — a compact high-capacity battery power station designed for home backup, camping, and mobile power needs. It includes multiple AC outlets, several USB-A and USB-C ports for charging laptops and phones, and a solar input for recharging via solar panels (sol

In [12]:
for deal in results.deals: 
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print("====================================================================")
    print("====================================================================")

Refurbished Apple iPhone 15 Pro Max (256GB) — a flagship-grade smartphone featuring Apple's latest design and high-end hardware in a refurbished condition. The 256GB capacity provides ample storage for photos, apps, and media, and the Pro Max model includes advanced camera systems, performance-optimized chipset, and a large high-resolution display. This unit is offered as a refurbished device, which typically includes inspection and restoration to working condition and may come with a limited warranty.
575.99
https://www.dealnews.com/Refurb-Apple-iPhones-at-eBay-Up-to-80-off-free-shipping/21802631.html?iref=rss-c142
EcoFlow Delta 2 Portable Power Station (1,024Wh) — a compact high-capacity battery power station designed for home backup, camping, and mobile power needs. It includes multiple AC outlets, several USB-A and USB-C ports for charging laptops and phones, and a solar input for recharging via solar panels (solar panels sold separately). The model (EFD330-2) supports AC charging 

In [13]:
root = logging.getLogger()
root.setLevel(logging.INFO)

## Scanner Agent

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()

agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing...
INFO:root:[Scanner Agent] Scanner Agent is set!
INFO:root:[Scanner Agent] Scanner Agent is fetching available deals...
INFO:root:[Scanner Agent] Scanner Agent received 30 new fresh deals!
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI client...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


DealSelection(deals=[Deal(product_description="Refurbished Apple iPhone 15 Pro Max with 256GB storage. This is a high-end iPhone model with a large OLED display, capable performance from Apple's A-series chipset, advanced camera system for pro-level photos and video, and generous onboard storage suitable for heavy app, photo and media use. The unit is refurbished and sold with free shipping.", price=575.99, url='https://www.dealnews.com/Refurb-Apple-iPhones-at-eBay-Up-to-80-off-free-shipping/21802631.html?iref=rss-c142'), Deal(product_description='Refurbished unlocked Apple iPhone 15 with 128GB storage. This unlocked smartphone offers current-generation performance, modern camera capabilities, and enough internal storage for everyday users. It comes refurbished with a 1-year Allstate warranty and ships free.', price=352.0, url='https://www.dealnews.com/Refurb-Unlocked-Apple-iPhone-15-128-GB-Smartphone-for-352-free-shipping/21802603.html?iref=rss-c142'), Deal(product_description="EcoFlo

## Using Pushover -- Mobile Device Notification

Pushover is a nifty tool for sending push notification to any mobile devices.

It's fairly easy to set up and install. 

A quick guideline is as down below:

1. Visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

2. Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like AIEngineer) and click Create Application.

3. Then add 2 lines to your .env file:

    PUSHOVER_USER = The User Key found on the right top corner of the homepage of PushOver

   PUSHOVER_TOKEN = The Token found right after clicking "Create an Application/API Token"

5. Finally, click "Add Phone" (otherwise Tablet or Desktop) to install on your phone.

In [30]:
load_dotenv(override=True)

True

In [31]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [32]:
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [47]:
def send_notification(msg): 

    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    payload = {
        "user": pushover_user, 
        "token": pushover_token, 
        "message": msg,
        "sound": "cashregister"
    }

    response = requests.post(pushover_url, data=payload, headers=headers)
    response.raise_for_status()

    return response

In [48]:
send_notification("Awesome deal found!")

<Response [200]>

## Messaging Agent

In [8]:
from agents.messaging_agent import MessagingAgent

In [9]:
agent = MessagingAgent()
agent.push("Such a massive deal!")

In [10]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com")